# <center>Data preprocessing</center>

- The structure of the data is to be changed to `[featurs, labels]` structures to be ably fine-tuned the model.

- The feature will be the input context which will accumulate 3 utterances before the answer.
- The labels will be the emotion of the context as well as the response. 
- At the end, we load the data into the Dataset class after applying 'arabert' preprocessing on context and emotion columns which will do all the preprocessing tasks on the Arabic sentences including the segmentation task.

## Install Needed libraries

In [1]:
# !pip install numpy
# !pip install pandas
# !pip install csv

## Import Needed Libraries

In [1]:
import os
import numpy as np
import pandas as pd

## Training data


read the cleaned training data

In [3]:
training_data = pd.read_csv("./Dataset/empatheticdialogues_arabic_cleaned/train_arabic_clean.csv")

structure the data into `["emotion","context","uterance"]` where the context contain 3 uterances before the responce on the same conversation

In [4]:
data_upd = []
history = []
max_hist_len = 3
for i in range(1, len(training_data)):
    cparts = training_data.iloc[i - 1]
    sparts = training_data.iloc[i]
    if cparts["conv_id"] == sparts["conv_id"]:
        prevsent = cparts["translated_utterance"]
        history.append(prevsent)
        idx = int(sparts["utterance_idx"])
        if ((idx % 2) == 0):
            context = " SOC ".join(history[-max_hist_len :])
            label = sparts["translated_utterance"]
            emotion = sparts["context"]
            data_upd.append([emotion,context,label])
    else:
        history = []

Write data to disk

In [5]:
path = "./Dataset/processed"
if(not os.path.isdir(path)):
    os.mkdir(path)
    
df_training = pd.DataFrame(data= data_upd,columns=["emotion","context","response"])
df_training.to_csv(path+"/train_arabic_with_history.csv",index=False)

In [6]:
print("Data Size: ", len(df_training))
df_training.sample(5)

Data Size:  37663


,emotion,context,response
23437,disappointed,كنت آمل حقًا أن أذهب إلى الحديقة المائية في نه...,هذا سيء. ربما يمكنك الذهاب في عطلة نهاية الأسب...
25997,disappointed,أنا محبط بعض الشيء. لقد تلقيت للتو مكالمة مفاد...,انا اكره ان اسمع ذالك. هل يستحق الشخص الذي حصل...
28175,furious,شخص ما اقتحم سيارتي ومزق كل شيء ، حتى السماعات...,أقترح نظام مراقبة Night Owl.
32960,faithful,أنا متزوج منذ 30 عامًا. أنا أحب زوجي ولكني أشت...,أوه ، ربما تكون هذه علامة لجلب المرح والمغامرة...
12232,lonely,أنا حقا قلت. أنا أفتقد نشاطي الشبابي. SOC قوتك...,حسنًا ، ربما تكون فرصة للتأمل. لكي تأخذ وقتًا ...


## Test data

In [7]:
test_data = pd.read_csv("./Dataset/empatheticdialogues_arabic_cleaned/test_arabic_clean.csv")

In [8]:
data_upd = []
history = []
max_hist_len = 3
for i in range(1, len(test_data)):
    cparts = test_data.iloc[i - 1]
    sparts = test_data.iloc[i]
    if cparts["conv_id"] == sparts["conv_id"]:
        prevsent = cparts["translated_utterance"]
        history.append(prevsent)
        idx = int(sparts["utterance_idx"])
        if ((idx % 2) == 0):
            context = " SOC ".join(history[-max_hist_len :])
            label = sparts["translated_utterance"]
            emotion = sparts["context"]
            data_upd.append([emotion,context,label])
    else:
        history = []

In [9]:
path = "./Dataset/processed"
if(not os.path.isdir(path)):
    os.mkdir(path)
    
df_test = pd.DataFrame(data= data_upd,columns=["emotion","context","response"])
df_test.to_csv(path+"/test_arabic_history.csv",index=False)

In [10]:
print("Data Length: ", len(df_test))
df_test.sample(5)

Data Length:  5247


,emotion,context,response
1529,lonely,شعرت بالعزلة الشديدة عندما انتقلنا لأول مرة إل...,واو ، لا أستطيع أن أتخيل كيف شعرت بالعيش في هو...
2519,apprehensive,أنا لا أثق في الناس بشكل عام. SOC لماذا لا تثق...,إنه محق. لكن يجب أن تثق في بعض الأشخاص الذين ت...
1451,content,أشعر بالراحة والسلام عندما أتأمل SOC من لا .. ...,سمعت أنه يمكنك السفر إلى أبعاد مختلفة أثناء ال...
2531,joyful,ذهبت مؤخرًا لأنابيب النهر مع صديقتي وأصدقائها ...,رائعة حقا! هل نزلت منحدرات وقمت بأشياء مخيفة؟
4324,trusting,أحب إقراض المال للأصدقاء. يمكن أن يفسد الصداقا...,كانت تلك مفاجأة جميلة! لقد حاولت إقراض المال ل...
